In [1]:
from tensorflow.examples.tutorials.mnist import  input_data
import numpy as np
import scipy.misc
import os
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
save_dir = "MNIST_data/raw"

if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

for i in range(50):
    image_array = mnist.train.images[i,:]
    image_array = image_array.reshape(28,28)
    filename = save_dir+"/mnist_train%d.jpg" % i
    scipy.misc.toimage(image_array, cmin=0.0 , cmax=1.0).save(filename)
print("Please check: %s" % save_dir)

Please check: MNIST_data/raw


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  # Remove the CWD from sys.path while we load stuff.


In [3]:
for i in range(5):
    label = mnist.train.labels[i,:]
    print(label)
    label = np.argmax(label)
    #print(label)
    print("mnist.train_%d.jpg label:%d" %(label, label))

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
mnist.train_7.jpg label:7
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
mnist.train_3.jpg label:3
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
mnist.train_4.jpg label:4
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
mnist.train_6.jpg label:6
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
mnist.train_1.jpg label:1


## sofmatmax分类

In [4]:
import tensorflow as tf
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,shape=[None,10])
w = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x,w)+b)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y)))
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cross_entropy)
train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y,1), tf.argmax(y_,1)),tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(1000):
    batch_x, batch_y = mnist.train.next_batch(batch_size=64)
    loss,_ = sess.run([cross_entropy,train_step], feed_dict={x: batch_x, y_: batch_y})
    if i % 500 == 0:
        acc = sess.run(train_acc, feed_dict={x: batch_x, y_:batch_y})
        print("Loss {:.2f}  accuracy {:.4f}".format(loss,acc))
print(sess.run(train_acc, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))  # 0.9185

Loss 147.37  accuracy 0.3594
Loss 28.12  accuracy 0.9219
0.916


### 双层卷积

In [18]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding="SAME")
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32,[None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])

#conv1
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#conv2
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#fc1+dropout
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#fc2
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2)+b_fc2
## 我们不采用先Softmax再计算交叉熵的方法，而是直接用tf.nn.softmax_cross_entropy_with_logits直接计算
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1)), tf.float32))
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


for i in range(1000):
    batch = mnist.train.next_batch(50)
    if i % 100 ==0:
        train_acc = sess.run(accuracy,feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d training accuracy: %g" % (i, train_acc))
    sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
#sess.close()


Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0 training accuracy: 0.08
step 100 training accuracy: 0.84
step 200 training accuracy: 0.88
step 300 training accuracy: 0.94
step 400 training accuracy: 0.92
step 500 training accuracy: 0.98
step 600 training accuracy: 0.92
step 700 training accuracy: 1
step 800 training accuracy: 0.92
step 900 training accuracy: 0.96


In [29]:
print(sess.run(accuracy,feed_dict={x: mnist.test.images[:500], y_: mnist.test.labels[:500], keep_prob: 1.0}))

0.962
